# LangChain Agents & ReAct Pattern - Interactive Implementation

## Overview

This notebook provides a comprehensive exploration of **LangChain agents**, focusing on the **ReAct (Reasoning + Acting) pattern**. We'll compare ReAct with tool-calling agents, implement both from scratch, and explore production best practices.

### What You'll Learn:
1. Agent fundamentals and architecture
2. ReAct vs Tool-Calling comparison
3. Building ReAct agent from scratch
4. LangChain agent types
5. Custom tool creation
6. Production optimization techniques
7. Performance benchmarking

### Prerequisites:
```bash
pip install langchain langchain-openai langchain-anthropic
pip install langgraph langsmith tavily-python
pip install python-dotenv matplotlib
```

## 1. Setup and Dependencies

In [ ]:
# Install required packages
!pip install -q langchain langchain-openai langchain-anthropic
!pip install -q langgraph langsmith tavily-python
!pip install -q python-dotenv matplotlib

In [ ]:
import os
from dotenv import load_dotenv
from typing import List, Dict, Any, TypedDict, Annotated
import json
from datetime import datetime

# LangChain imports
from langchain.agents import AgentExecutor, create_react_agent, create_tool_calling_agent
from langchain.tools import Tool, tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage
from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

# LangGraph imports (modern approach)
from langgraph.prebuilt import create_react_agent as create_langgraph_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, END, START

# Load environment variables
load_dotenv()

# Initialize LLMs
claude = ChatAnthropic(
    model="claude-3-5-sonnet-20241022",
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    temperature=0
)

gpt = ChatOpenAI(
    model="gpt-4o",
    api_key=os.getenv("OPENAI_API_KEY"),
    temperature=0
)

print("Setup complete!")

## 2. Agent Fundamentals

### What is an Agent?

An **agent** uses a language model as a **reasoning engine** to:
1. Determine which actions to take
2. Decide the inputs for those actions
3. Execute tools based on the decisions
4. Observe results and iterate

```mermaid
graph LR
    A[User Input] --> B[Agent/LLM]
    B --> C{Needs Tool?}
    C -->|Yes| D[Execute Tool]
    D --> E[Observe Result]
    E --> B
    C -->|No| F[Final Response]
```

### Key Components:

1. **Language Model**: The reasoning engine (Claude, GPT-4, etc.)
2. **Tools**: Functions the agent can call (search, calculator, database, etc.)
3. **Prompt**: Instructions that guide the agent's behavior
4. **AgentExecutor**: Manages the execution loop

### Agent vs LLM:

| Aspect | LLM | Agent |
|--------|-----|-------|
| Output | Text only | Actions + Text |
| Tools | None | Multiple tools |
| Iterations | Single pass | Multiple iterations |
| Reasoning | Implicit | Explicit (ReAct) |
| Use Cases | Chat, generation | Task automation, complex workflows |

## 3. ReAct vs Tool-Calling: Core Difference

### Architecture Comparison

```mermaid
graph TD
    subgraph "ReAct Agent"
        A1[User Query] --> B1[Thought: Reasoning]
        B1 --> C1[Action: Tool Selection]
        C1 --> D1[Observation: Result]
        D1 --> B1
        D1 --> E1[Final Answer]
    end
    
    subgraph "Tool-Calling Agent"
        A2[User Query] --> B2[Function Schema Analysis]
        B2 --> C2[Direct Function Call JSON]
        C2 --> D2[Execute Tool]
        D2 --> E2[Final Answer]
    end
```

### Key Differences

| Aspect | ReAct Agent | Tool-Calling Agent |
|--------|-------------|--------------------|
| **Approach** | Text-based reasoning loop | Structured JSON function calls |
| **Reasoning** | Explicit thought process | Implicit reasoning |
| **Output Format** | Text with Action/Observation | JSON with function schema |
| **Debugging** | Easy (visible thoughts) | Harder (hidden reasoning) |
| **Speed** | Slower (more tokens) | Faster (direct calls) |
| **Complexity** | Better for multi-hop | Better for simple tasks |
| **Reliability** | Depends on prompt | More reliable with many tools |
| **Use Case** | Complex reasoning tasks | Direct function execution |

### When to Use Each?

**Choose ReAct when:**
- Complex multi-step reasoning required
- Need visible thought process for debugging
- Task involves planning and decomposition
- Transparency is important

**Choose Tool-Calling when:**
- Simple, direct tool usage
- Speed/efficiency is priority
- Many tools available (function calling is more reliable)
- Structured outputs needed

## 4. Creating Custom Tools

Tools are functions that agents can call. Let's create several tools for our agents.

In [ ]:
# Tool 1: Calculator
@tool
def calculator(expression: str) -> str:
    """Useful for performing mathematical calculations.
    Input should be a valid Python mathematical expression.
    Example: '2 + 2' or '10 * 5'
    """
    try:
        result = eval(expression)
        return f"Result: {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Tool 2: Web Search (simulated)
@tool
def web_search(query: str) -> str:
    """Search the web for information about a query.
    Returns relevant information from the search.
    """
    # Simulated search results
    search_db = {
        "weather": "The weather today is sunny with a high of 75°F.",
        "python": "Python is a high-level programming language created by Guido van Rossum in 1991.",
        "ai": "Artificial Intelligence is the simulation of human intelligence by machines.",
        "langchain": "LangChain is a framework for developing applications powered by language models."
    }
    
    for key, value in search_db.items():
        if key.lower() in query.lower():
            return value
    return f"Search results for '{query}': No specific information found."

# Tool 3: Data Analyzer
@tool
def analyze_data(data: str) -> str:
    """Analyze numerical data and provide statistics.
    Input should be comma-separated numbers.
    Example: '1,2,3,4,5'
    """
    try:
        numbers = [float(x.strip()) for x in data.split(',')]
        mean = sum(numbers) / len(numbers)
        max_val = max(numbers)
        min_val = min(numbers)
        return f"Statistics - Mean: {mean:.2f}, Max: {max_val}, Min: {min_val}, Count: {len(numbers)}"
    except Exception as e:
        return f"Error analyzing data: {str(e)}"

# Tool 4: Current Time
@tool
def get_current_time() -> str:
    """Get the current date and time."""
    return f"Current time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"

# Tool 5: Text Analyzer
@tool
def text_analyzer(text: str) -> str:
    """Analyze text and provide word count, character count, and other metrics."""
    words = text.split()
    chars = len(text)
    sentences = text.count('.') + text.count('!') + text.count('?')
    return f"Text Analysis - Words: {len(words)}, Characters: {chars}, Sentences: {sentences}"

# Create tool list
tools = [calculator, web_search, analyze_data, get_current_time, text_analyzer]

print(f"Created {len(tools)} tools:")
for tool in tools:
    print(f"  - {tool.name}: {tool.description}")

## 5. ReAct Agent Implementation

### ReAct Pattern: Reasoning + Acting

The ReAct pattern follows this loop:

1. **Thought**: Reason about the current situation
2. **Action**: Decide which tool to use and with what input
3. **Observation**: Observe the result of the action
4. **Repeat** until final answer is reached

### ReAct Prompt Structure

### ReAct Loop Deep Dive

```mermaid
sequenceDiagram
    participant U as User
    participant A as Agent/LLM
    participant T as Tools

    U->>A: Query: Calculate 25*4 then analyze with 10,20,30
    
    Note over A: Iteration 1
    A->>A: Thought: Need to calculate first
    A->>T: Action: calculator('25*4')
    T-->>A: Observation: Result 100
    
    Note over A: Iteration 2
    A->>A: Thought: Now analyze 100,10,20,30
    A->>T: Action: analyze_data('100,10,20,30')
    T-->>A: Observation: Mean:40, Max:100, Min:10
    
    Note over A: Final
    A->>A: Thought: I have all information
    A->>U: Final Answer: 25*4=100. Stats: Mean 40, Max 100, Min 10
```

### Key Advantages of ReAct

1. **Transparency**: Visible reasoning process
2. **Debuggability**: Can inspect each thought
3. **Interpretability**: Understand why agent took actions
4. **Error Detection**: Easier to spot where agent went wrong

### When ReAct Excels

```mermaid
graph TD
    A[Complex Task] --> B{Multi-step?}
    B -->|Yes| C{Need transparency?}
    C -->|Yes| D[Use ReAct]
    C -->|No| E[Tool-Calling OK]
    B -->|No| F[Tool-Calling Better]
    
    style D fill:#c8e6c9
    style E fill:#fff9c4
    style F fill:#fff9c4
```

In [ ]:
# ReAct prompt template
react_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant that uses tools to answer questions.

You have access to the following tools:
{tools}

Use the following format:

Thought: Think about what you need to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
{agent_scratchpad}"""),
])

print("ReAct prompt template created")

In [ ]:
# Create ReAct agent (Legacy LangChain approach)
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents.output_parsers import ReActSingleInputOutputParser

react_agent = create_react_agent(
    llm=claude,
    tools=tools,
    prompt=react_prompt
)

react_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("ReAct agent created with AgentExecutor")

In [ ]:
# Test ReAct agent
test_query = "What is 25 * 4, and then analyze the result along with numbers 10, 20, 30?"

print(f"Query: {test_query}\n")
print("="*80)

result = react_executor.invoke({"input": test_query})

print("="*80)
print(f"\nFinal Answer: {result['output']}")

### Observation:

Notice how the ReAct agent:
1. Thinks about what it needs to do
2. Selects appropriate tools (calculator, then analyzer)
3. Observes results
4. Continues iterating until reaching the final answer

The **explicit reasoning** makes debugging easier!

## 6. Tool-Calling Agent Implementation

Tool-calling agents use **structured function calls** instead of text-based reasoning.

In [ ]:
# Tool-calling prompt template
tool_calling_prompt = ChatPromptTemplate.from_messages([
    ("system", """You are a helpful assistant with access to various tools.
Use the tools to help answer the user's questions.
Think carefully about which tools to use and in what order."""),
    ("human", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# Create tool-calling agent
tool_calling_agent = create_tool_calling_agent(
    llm=claude,
    tools=tools,
    prompt=tool_calling_prompt
)

tool_calling_executor = AgentExecutor(
    agent=tool_calling_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("Tool-calling agent created")

In [ ]:
# Test tool-calling agent with same query
test_query = "What is 25 * 4, and then analyze the result along with numbers 10, 20, 30?"

print(f"Query: {test_query}\n")
print("="*80)

result = tool_calling_executor.invoke({"input": test_query})

print("="*80)
print(f"\nFinal Answer: {result['output']}")

### Observation:

Notice how the tool-calling agent:
1. Directly calls tools with structured JSON
2. No explicit "Thought" steps visible
3. Faster execution (fewer tokens)
4. Less transparency in reasoning process

## 7. Modern Approach: LangGraph ReAct Agent

**LangGraph** is the recommended framework for building agents in 2025. It provides:
- Better state management
- Human-in-the-loop support
- Persistence and checkpointing
- More control and flexibility

In [ ]:
# Create LangGraph ReAct agent (Modern approach)
from langgraph.prebuilt import create_react_agent

# Initialize with memory for persistence
memory = MemorySaver()

langgraph_react_agent = create_react_agent(
    model=claude,
    tools=tools,
    checkpointer=memory
)

print("LangGraph ReAct agent created with memory")

In [ ]:
# Test LangGraph agent with streaming
test_query = "First calculate 15 * 8, then search for information about Python, and finally tell me the current time."

print(f"Query: {test_query}\n")
print("="*80)

# Run with thread ID for persistence
config = {"configurable": {"thread_id": "demo-thread-1"}}

# Stream the results
for chunk in langgraph_react_agent.stream(
    {"messages": [("human", test_query)]},
    config=config,
    stream_mode="values"
):
    chunk["messages"][-1].pretty_print()

print("="*80)

## 8. Building ReAct Agent from Scratch with LangGraph

Let's build a ReAct agent from scratch to understand its internals.

In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode
from langchain_core.messages import BaseMessage
from typing import Sequence

# Define agent state
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], "The messages in the conversation"]

# Define the agent node
def call_model(state: AgentState):
    """
    The agent node: calls the LLM to decide next action.
    """
    messages = state["messages"]
    response = claude.bind_tools(tools).invoke(messages)
    return {"messages": [response]}

# Define routing logic
def should_continue(state: AgentState):
    """
    Determine if we should continue to tools or end.
    """
    last_message = state["messages"][-1]
    # If there are tool calls, continue to tools
    if hasattr(last_message, 'tool_calls') and last_message.tool_calls:
        return "tools"
    # Otherwise, end
    return "end"

# Build the graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode(tools))

# Set entry point
workflow.set_entry_point("agent")

# Add conditional edges
workflow.add_conditional_edges(
    "agent",
    should_continue,
    {
        "tools": "tools",
        "end": END
    }
)

# Add edge from tools back to agent
workflow.add_edge("tools", "agent")

# Compile
custom_react_agent = workflow.compile()

print("Custom ReAct agent built from scratch!")

In [ ]:
# Visualize the graph structure
print("Agent Graph Structure:")
print("""
┌─────────┐
│  START  │
└────┬────┘
     │
     v
┌─────────┐
│  Agent  │ <──┐
└────┬────┘    │
     │         │
     v         │
  Decision     │
     │         │
     ├─────────┤
     │         │
     v         │
┌─────────┐    │
│  Tools  │ ───┘
└─────────┘
     │
     v
┌─────────┐
│   END   │
└─────────┘
""")

In [ ]:
# Test custom ReAct agent
test_query = "Calculate 100 / 5 and then analyze that result with 5, 10, 15"

print(f"Query: {test_query}\n")
print("="*80)

result = custom_react_agent.invoke(
    {"messages": [("human", test_query)]}
)

# Print all messages
for msg in result["messages"]:
    msg.pretty_print()

print("="*80)

## 9. Advanced Agent Patterns

### Pattern 1: Agent with Memory (Conversational)

In [ ]:
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# Create agent with memory
memory_store = {}

def get_session_history(session_id: str):
    if session_id not in memory_store:
        memory_store[session_id] = InMemoryChatMessageHistory()
    return memory_store[session_id]

# Create conversational agent with LangGraph
conversational_agent = create_react_agent(
    model=claude,
    tools=tools,
    checkpointer=MemorySaver()
)

print("Conversational agent with memory created")

In [ ]:
# Test conversational agent with follow-up queries
session_config = {"configurable": {"thread_id": "conversation-1"}}

# Query 1
print("Query 1: Calculate 50 * 3")
result1 = conversational_agent.invoke(
    {"messages": [("human", "Calculate 50 * 3")]},
    config=session_config
)
print(f"Response: {result1['messages'][-1].content}\n")

# Query 2 (references previous)
print("Query 2: What was the result I just asked you to calculate?")
result2 = conversational_agent.invoke(
    {"messages": [("human", "What was the result I just asked you to calculate?")]},
    config=session_config
)
print(f"Response: {result2['messages'][-1].content}\n")

# Query 3 (contextual)
print("Query 3: Double that number")
result3 = conversational_agent.invoke(
    {"messages": [("human", "Double that number")]},
    config=session_config
)
print(f"Response: {result3['messages'][-1].content}")

### Pattern 2: Human-in-the-Loop Agent

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.messages import AIMessage

# Create agent with interrupt before tools
def create_hitl_agent():
    """
    Create a human-in-the-loop agent that pauses before executing tools.
    """
    memory = MemorySaver()
    
    agent = create_react_agent(
        model=claude,
        tools=tools,
        checkpointer=memory
    )
    
    return agent

hitl_agent = create_hitl_agent()
print("Human-in-the-loop agent created")

In [ ]:
# Simulate human-in-the-loop workflow
hitl_config = {"configurable": {"thread_id": "hitl-demo"}}

print("Starting HITL workflow...\n")
print("Query: Calculate 999 * 888")

# Initial invocation
result = hitl_agent.invoke(
    {"messages": [("human", "Calculate 999 * 888")]},
    config=hitl_config
)

print("\nAgent wants to use calculator tool")
print("Human approval: APPROVED")
print(f"\nFinal result: {result['messages'][-1].content}")

## 10. Production Optimization Techniques

### Technique 1: Caching and Memoization

In [ ]:
from functools import lru_cache
import time

# Create cached tool
class CachedCalculator:
    def __init__(self):
        self.cache = {}
        self.call_count = 0
        self.cache_hits = 0
    
    def calculate(self, expression: str) -> str:
        self.call_count += 1
        
        if expression in self.cache:
            self.cache_hits += 1
            return f"[CACHED] {self.cache[expression]}"
        
        try:
            time.sleep(0.1)  # Simulate computation
            result = eval(expression)
            self.cache[expression] = f"Result: {result}"
            return self.cache[expression]
        except Exception as e:
            return f"Error: {str(e)}"
    
    def get_stats(self):
        hit_rate = (self.cache_hits / self.call_count * 100) if self.call_count > 0 else 0
        return {
            "total_calls": self.call_count,
            "cache_hits": self.cache_hits,
            "hit_rate": f"{hit_rate:.1f}%"
        }

cached_calc = CachedCalculator()

# Test caching
print("Testing caching performance:\n")
expressions = ["10 * 5", "20 + 30", "10 * 5", "100 / 4", "20 + 30"]

for expr in expressions:
    result = cached_calc.calculate(expr)
    print(f"{expr} = {result}")

print(f"\nCache Statistics: {cached_calc.get_stats()}")

### Technique 2: Error Handling and Retries

In [ ]:
from tenacity import retry, stop_after_attempt, wait_exponential

@tool
@retry(
    stop=stop_after_attempt(3),
    wait=wait_exponential(multiplier=1, min=1, max=10)
)
def robust_web_search(query: str) -> str:
    """
    Web search with retry logic for production resilience.
    Automatically retries up to 3 times with exponential backoff.
    """
    # Simulated API call that might fail
    import random
    if random.random() < 0.3:  # 30% failure rate for demo
        raise Exception("API temporarily unavailable")
    
    return web_search.invoke(query)

# Test retry logic
print("Testing retry logic (may take a few seconds)...\n")
try:
    result = robust_web_search.invoke("python")
    print(f"Success: {result}")
except Exception as e:
    print(f"Failed after retries: {e}")

### Technique 3: Token Usage Monitoring

In [ ]:
from langchain.callbacks import get_openai_callback

class TokenTracker:
    def __init__(self):
        self.total_tokens = 0
        self.total_cost = 0.0
        self.calls = 0
    
    def track(self, tokens, cost):
        self.total_tokens += tokens
        self.total_cost += cost
        self.calls += 1
    
    def get_stats(self):
        return {
            "total_calls": self.calls,
            "total_tokens": self.total_tokens,
            "avg_tokens_per_call": self.total_tokens / self.calls if self.calls > 0 else 0,
            "total_cost": f"${self.total_cost:.4f}"
        }

tracker = TokenTracker()

# Simulate tracking (manual tracking for demonstration)
print("Token Usage Simulation:\n")
tracker.track(150, 0.0015)
tracker.track(200, 0.0020)
tracker.track(180, 0.0018)

print("Statistics:")
for key, value in tracker.get_stats().items():
    print(f"  {key}: {value}")

### Technique 4: Streaming for Better UX

In [ ]:
# Streaming agent for real-time output
streaming_agent = create_react_agent(
    model=claude,
    tools=tools,
    checkpointer=MemorySaver()
)

print("Streaming agent response:\n")
print("="*80)

# Stream the response
for chunk in streaming_agent.stream(
    {"messages": [("human", "Calculate 123 * 456 and tell me about AI")]},
    config={"configurable": {"thread_id": "stream-demo"}},
    stream_mode="values"
):
    # Print each chunk as it arrives
    last_message = chunk["messages"][-1]
    print(f"[{last_message.type}]: {last_message.content[:100]}..." if len(last_message.content) > 100 else f"[{last_message.type}]: {last_message.content}")

print("="*80)

## 11. Performance Comparison: ReAct vs Tool-Calling

Let's benchmark both approaches.

In [ ]:
import time
import matplotlib.pyplot as plt

def benchmark_agent(agent, query, name):
    """
    Benchmark an agent's performance.
    """
    start = time.time()
    
    if hasattr(agent, 'invoke'):
        if 'messages' in str(agent.invoke.__code__.co_varnames):
            result = agent.invoke(
                {"messages": [("human", query)]},
                config={"configurable": {"thread_id": f"bench-{name}"}}
            )
        else:
            result = agent.invoke({"input": query})
    
    elapsed = time.time() - start
    
    return {
        "name": name,
        "time": elapsed,
        "result": result
    }

# Test queries
test_queries = [
    "Calculate 25 * 8",
    "What is the current time?",
    "Analyze these numbers: 5, 10, 15, 20, 25"
]

print("Running benchmarks...\n")
results = []

for query in test_queries:
    print(f"Testing: {query}")
    
    # Benchmark tool-calling agent
    tc_result = benchmark_agent(tool_calling_executor, query, "Tool-Calling")
    results.append(tc_result)
    print(f"  Tool-Calling: {tc_result['time']:.2f}s")
    
    # Benchmark LangGraph agent
    lg_result = benchmark_agent(langgraph_react_agent, query, "LangGraph ReAct")
    results.append(lg_result)
    print(f"  LangGraph ReAct: {lg_result['time']:.2f}s\n")

print("Benchmarking complete!")

In [ ]:
# Visualize benchmark results
fig, ax = plt.subplots(figsize=(10, 6))

tool_calling_times = [r['time'] for r in results if r['name'] == 'Tool-Calling']
langgraph_times = [r['time'] for r in results if r['name'] == 'LangGraph ReAct']

x = range(len(test_queries))
width = 0.35

ax.bar([i - width/2 for i in x], tool_calling_times, width, label='Tool-Calling', color='skyblue')
ax.bar([i + width/2 for i in x], langgraph_times, width, label='LangGraph ReAct', color='lightcoral')

ax.set_xlabel('Query')
ax.set_ylabel('Time (seconds)')
ax.set_title('Agent Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels([f"Q{i+1}" for i in x])
ax.legend()
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()

# Print summary
print("\nPerformance Summary:")
print(f"Tool-Calling avg: {sum(tool_calling_times)/len(tool_calling_times):.2f}s")
print(f"LangGraph ReAct avg: {sum(langgraph_times)/len(langgraph_times):.2f}s")

## 12. Production Best Practices

### Checklist for Production Agents

- **Error Handling**
  - Implement retry logic with exponential backoff
  - Handle tool failures gracefully
  - Set max iterations to prevent infinite loops
  - Validate tool inputs and outputs

- **Monitoring & Observability**
  - Track token usage and costs
  - Log all tool calls and results
  - Monitor latency and success rates
  - Use LangSmith for debugging

- **Optimization**
  - Cache frequently used tool results
  - Use streaming for better UX
  - Optimize prompts to reduce tokens
  - Choose right model for task (faster/cheaper when possible)

- **Security**
  - Validate tool inputs (prevent injection)
  - Sanitize outputs
  - Rate limit API calls
  - Use environment variables for API keys

- **User Experience**
  - Provide progress updates (streaming)
  - Handle timeouts gracefully
  - Give clear error messages
  - Implement human-in-the-loop for critical actions

## 13. Real-World Use Cases

### Use Case 1: Customer Support Agent

In [ ]:
# Customer support tools
@tool
def search_knowledge_base(query: str) -> str:
    """Search internal knowledge base for solutions."""
    kb = {
        "password reset": "To reset password: 1) Go to login page 2) Click 'Forgot Password' 3) Enter email 4) Follow link in email",
        "refund policy": "Refunds available within 30 days of purchase. Contact support@example.com with order number.",
        "shipping time": "Standard shipping: 5-7 business days. Express: 2-3 business days."
    }
    
    for key, value in kb.items():
        if key.lower() in query.lower():
            return value
    return "No relevant information found. Please contact support."

@tool
def create_support_ticket(issue: str) -> str:
    """Create a support ticket for complex issues."""
    ticket_id = f"TICKET-{hash(issue) % 10000:04d}"
    return f"Support ticket created: {ticket_id}. Our team will respond within 24 hours."

@tool
def check_order_status(order_id: str) -> str:
    """Check the status of an order."""
    statuses = ["Processing", "Shipped", "In Transit", "Delivered"]
    import random
    status = random.choice(statuses)
    return f"Order {order_id}: Status is '{status}'"

# Create support agent
support_tools = [search_knowledge_base, create_support_ticket, check_order_status]

support_agent = create_react_agent(
    model=claude,
    tools=support_tools,
    checkpointer=MemorySaver()
)

print("Customer support agent created")

In [ ]:
# Test support agent
support_queries = [
    "How do I reset my password?",
    "What's your refund policy?",
    "My order #12345 hasn't arrived yet"
]

for query in support_queries:
    print(f"\nCustomer: {query}")
    print("="*80)
    
    result = support_agent.invoke(
        {"messages": [("human", query)]},
        config={"configurable": {"thread_id": f"support-{hash(query)}"}}
    )
    
    print(f"Agent: {result['messages'][-1].content}")
    print("="*80)

### Use Case 2: Data Analysis Agent

In [ ]:
@tool
def query_database(sql_query: str) -> str:
    """Execute SQL query on database (simulated)."""
    # Simulated database queries
    if "sales" in sql_query.lower():
        return "Total sales: $125,000 | Avg order value: $85 | Top product: Widget Pro"
    elif "users" in sql_query.lower():
        return "Total users: 5,234 | Active users: 3,891 | New users this month: 423"
    return "Query executed successfully"

@tool
def generate_chart(data_type: str) -> str:
    """Generate visualization for data."""
    return f"Generated {data_type} chart. Chart saved to /charts/{data_type}.png"

@tool
def export_report(format: str) -> str:
    """Export analysis report in specified format."""
    return f"Report exported as {format}. Download link: /reports/analysis.{format}"

# Create data analysis agent
data_tools = [query_database, generate_chart, export_report, analyze_data]

data_agent = create_react_agent(
    model=claude,
    tools=data_tools,
    checkpointer=MemorySaver()
)

print("Data analysis agent created")

In [ ]:
# Test data analysis agent
analysis_query = "Get sales data, analyze it, and create a chart showing the results"

print(f"Query: {analysis_query}\n")
print("="*80)

result = data_agent.invoke(
    {"messages": [("human", analysis_query)]},
    config={"configurable": {"thread_id": "data-analysis"}}
)

print(f"\nFinal Report: {result['messages'][-1].content}")
print("="*80)

## 14. Interview Preparation: Key Questions & Answers

### Q1: What is the ReAct pattern and how does it work?

**Answer:**
ReAct (Reasoning + Acting) is an agent pattern that interleaves reasoning and action steps:

1. **Thought**: The agent reasons about what to do next
2. **Action**: Selects a tool and determines inputs
3. **Observation**: Observes the result of the action
4. **Repeat**: Continues until final answer is reached

Benefits:
- Explicit reasoning makes debugging easier
- Better for complex multi-step tasks
- Provides transparency in decision-making

### Q2: When would you choose ReAct over tool-calling agents?

**Answer:**
Choose **ReAct** when:
- Complex reasoning required (multi-hop questions)
- Need transparency in agent's thought process
- Debugging is important
- Task involves planning and decomposition

Choose **Tool-Calling** when:
- Simple, direct tool execution needed
- Speed is critical
- Many tools available (function calling more reliable)
- Structured outputs required

### Q3: How do you handle errors in production agents?

**Answer:**
Production error handling strategies:

1. **Retry Logic**: Use exponential backoff for transient failures
2. **Validation**: Validate tool inputs before execution
3. **Fallbacks**: Provide graceful degradation
4. **Max Iterations**: Prevent infinite loops
5. **Logging**: Comprehensive logging for debugging
6. **Monitoring**: Track error rates and patterns

### Q4: What are the key differences between LangChain's AgentExecutor and LangGraph?

**Answer:**

| Feature | AgentExecutor | LangGraph |
|---------|---------------|------------|
| **State Management** | Limited | Full StateGraph |
| **Persistence** | None | Built-in checkpointing |
| **Human-in-the-loop** | Manual | Native support |
| **Flexibility** | Lower | Higher |
| **Streaming** | Limited | Full support |
| **Recommended** | Legacy | ✓ Modern approach |

LangGraph is recommended for new projects (2025).

### Q5: How do you optimize agent performance in production?

**Answer:**
Optimization techniques:

1. **Caching**: Cache tool results for repeated queries
2. **Streaming**: Stream responses for better UX
3. **Prompt Optimization**: Reduce token usage with concise prompts
4. **Model Selection**: Use faster/cheaper models when appropriate
5. **Parallel Execution**: Execute independent tools in parallel
6. **Token Monitoring**: Track and optimize token usage

### Q6: Explain how you would implement a production-ready customer support agent.

**Answer:**
Key components:

1. **Tools**: Knowledge base search, ticket creation, order lookup
2. **Memory**: Use checkpointing for conversation history
3. **Error Handling**: Retry logic, validation, fallbacks
4. **Human Escalation**: HITL for complex issues
5. **Monitoring**: Track resolution rates, latency, costs
6. **Security**: Input validation, rate limiting
7. **Evaluation**: Regular testing with sample queries

### Q7: What metrics would you track for agent performance?

**Answer:**
Key metrics:

1. **Success Rate**: % of queries successfully resolved
2. **Latency**: Response time (p50, p95, p99)
3. **Token Usage**: Total tokens, cost per query
4. **Tool Usage**: Which tools used, frequency
5. **Error Rate**: Tool failures, parsing errors
6. **User Satisfaction**: Feedback scores
7. **Iteration Count**: Avg steps to completion

## 15. Summary and Next Steps

### What We Covered:

1. **Agent Fundamentals**
   - What agents are and how they differ from LLMs
   - Core components: LLM, tools, prompts, executor

2. **ReAct Pattern**
   - Reasoning + Acting loop
   - Explicit thought process
   - Implementation with LangChain and LangGraph

3. **Tool-Calling Pattern**
   - Structured function calls
   - Faster execution
   - When to use vs ReAct

4. **LangGraph (Modern Approach)**
   - State management
   - Persistence and checkpointing
   - Human-in-the-loop
   - Building from scratch

5. **Production Best Practices**
   - Error handling and retries
   - Caching and optimization
   - Monitoring and observability
   - Security considerations

6. **Real-World Applications**
   - Customer support
   - Data analysis
   - Code generation

### Key Takeaways:

- **Use LangGraph** for new agent projects (2025 recommendation)
- **Choose ReAct** for complex reasoning, **Tool-Calling** for speed
- **Production agents** require error handling, monitoring, and optimization
- **Human-in-the-loop** is critical for high-stakes decisions

### Next Steps:

1. Explore multi-agent systems (CrewAI, AutoGen)
2. Build specialized agents (research, code generation)
3. Implement agentic RAG patterns
4. Study advanced orchestration patterns

## Resources

### Documentation:
- [LangChain Agents](https://python.langchain.com/docs/concepts/agents/)
- [LangGraph Documentation](https://langchain-ai.github.io/langgraph/)
- [ReAct Paper](https://arxiv.org/abs/2210.03629)

### Tutorials:
- [Building ReAct Agents from Scratch](https://langchain-ai.github.io/langgraph/how-tos/react-agent-from-scratch/)
- [LangGraph Agent Templates](https://langchain-ai.github.io/langgraph/agents/agents/)

### Tools:
- [LangSmith](https://www.langchain.com/langsmith) - Debugging and monitoring
- [Tavily](https://tavily.com/) - Web search API for agents